##### To filter the cash flow of each stock for the record closest to the IPO date.

In [1]:
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
d_parser = lambda x: pd.datetime.strptime(x,'%Y-%m-%d')
IPO_df = pd.read_csv('raw_dataset/IPO_Listing_2019_2020.csv', parse_dates = ['IPO_DATE'], date_parser = d_parser)
IPO_df

C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


,IPO_DATE,SYMBOL,NAME,IPO_PRICE,CURRENT,RETURN
0,2020-12-29,MRM,Medirom Healthcare,15.0,10.30,-0.3133
1,2020-12-24,VTAQ,Ventoux CCM Acquisition,10.0,9.89,-0.0110
2,2020-12-23,IKT,Inhibikase Therapeutics,10.0,6.21,-0.3790
3,2020-12-23,GBS,"GBS, Inc.",17.0,6.81,-0.5994
4,2020-12-23,HCAR,Healthcare Services Acquisition,10.0,9.85,-0.0155
...,...,...,...,...,...,...
705,2019-01-17,RBKB,Rhinebeck Bancorp,10.0,10.49,0.0490
706,2019-01-09,BCOW,1895 Bancorp of Wisconsin,10.0,14.94,0.4940
707,2019-01-08,MDJH,MDJM Ltd.,5.0,4.25,-0.1500
708,2019-01-08,MTC,"MMTec, Inc.",4.0,2.83,-0.2925


In [3]:
IPO_df.dtypes

IPO_DATE     datetime64[ns]
SYMBOL               object
NAME                 object
IPO_PRICE           float64
CURRENT             float64
RETURN              float64
dtype: object

In [4]:
IPO_list = IPO_df['SYMBOL'].tolist()
IPO_list

['MRM',
 'VTAQ',
 'IKT',
 'GBS',
 'HCAR',
 'CFIV',
 'VII',
 'VHAQ',
 'ACKIT',
 'SVOK',
 'GFX',
 'MASS',
 'DUNE',
 'MTAC',
 'IIII',
 'MDWT',
 'SCOA',
 'COOL',
 'VIRI',
 'WISH',
 'UPST',
 'CCV',
 'SCPS',
 'BCAB',
 'WNW',
 'KINZ',
 'MRAC',
 'OCG',
 'BLUW',
 'ATA',
 'ABCL',
 'NBTX',
 'FDMT',
 'DWIN',
 'ROCC',
 'GLAQ',
 'MOTV',
 'EDTX',
 'SNRH',
 'GHVI',
 'TVAC',
 'CBAH',
 'VVOS',
 'CERT',
 'RMGB',
 'HYFM',
 'ABNB',
 'CTAQ',
 'PCPC',
 'NEBC',
 'DASH',
 'HMCO',
 'PUBM',
 'ALTU',
 'AI',
 'FLAC',
 'CND',
 'DDMX',
 'MUDS',
 'RAAC',
 'YQ',
 'SBTX',
 'SEER',
 'SGTX',
 'FTCV',
 'PTIC',
 'FPAC',
 'HTPA',
 'KNTE',
 'LOKB',
 'DCBO',
 'SPFR',
 'RSVA',
 'TACA',
 'CAP',
 'SGAM',
 'FRX',
 'SPRQ',
 'HFEN',
 'OZON',
 'SV',
 'VMAR',
 'GNPK',
 'VCVC',
 'TINV',
 'BREZ',
 'LSAQ',
 'ARBG',
 'MRVI',
 'CAS',
 'SHC',
 'KWAC',
 'OCA',
 'IIAC',
 'NGMS',
 'LNFA',
 'RTPZ',
 'TLS',
 'YSG',
 'OLMA',
 'JYAC',
 'CHFW',
 'PHIC',
 'DGNS',
 'PIPP',
 'OTRA',
 'ZNTE',
 'HAAC',
 'CFAC',
 'BWAC',
 'DMYI',
 'STIC',
 'NOAC',
 'TSI

In [5]:
# Check length of IPO_list
len(IPO_list)

710

#### Create a loop to perform a filter on each statement for each stock

In [6]:
# Loop through all symbols in IPO_List

FileNotFoundError_List = []

for symbol in IPO_list:
    
    
    try: 

        symbol_df = IPO_df.loc[IPO_df['SYMBOL']==symbol]
        symbol_IPO_DATE = symbol_df['IPO_DATE']

        # Define range lower bound and upper bound
        delta_days = 45

        date_lower_bound = symbol_IPO_DATE - timedelta(days=delta_days)
        date_upper_bound = symbol_IPO_DATE + timedelta(days=delta_days)

        print(f'{symbol}: {symbol_IPO_DATE}')
        print(f'{symbol} lower bound: {date_lower_bound}')
        print(f'{symbol} upper bound: {date_upper_bound}')
        print(f'----------------------------------')

        # Define dynamic path for each symbol
        symbol_cash_flow_path = f'raw_dataset/cash_flow/{symbol}_cash_flow.csv'

        # Read in each symbol's income statement
        d_parser = lambda x: pd.datetime.strptime(x,'%Y-%m-%d')
        symbol_cash_flow_df = pd.read_csv(symbol_cash_flow_path, parse_dates = ['fiscalDateEnding'], date_parser = d_parser, index_col=0)
        pd.set_option('display.max_columns',85)

        # To filter for dates within the IPO_date +/- 30 days range
        fiscalDate_filt = (symbol_cash_flow_df['fiscalDateEnding'] >= date_lower_bound.squeeze()) & (symbol_cash_flow_df['fiscalDateEnding'] <= date_upper_bound.squeeze())
        filtered_symbol_cash_flow_df = symbol_cash_flow_df.loc[fiscalDate_filt]

        # Exporting test income statement dataframe to csv file
        save_file_path = f'raw_dataset/filtered_cash_flow/{symbol}_filtered_cash_flow.csv'
        filtered_symbol_cash_flow_df.to_csv(save_file_path, index=False)
        
    except FileNotFoundError:
        print(f"FileNotFoundError occured on {symbol}")
        print(f'----------------------------------')
        FileNotFoundError_List.append(symbol)


C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


MRM: 0   2020-12-29
Name: IPO_DATE, dtype: datetime64[ns]
MRM lower bound: 0   2020-11-14
Name: IPO_DATE, dtype: datetime64[ns]
MRM upper bound: 0   2021-02-12
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
VTAQ: 1   2020-12-24
Name: IPO_DATE, dtype: datetime64[ns]
VTAQ lower bound: 1   2020-11-09
Name: IPO_DATE, dtype: datetime64[ns]
VTAQ upper bound: 1   2021-02-07
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
IKT: 2   2020-12-23
Name: IPO_DATE, dtype: datetime64[ns]
IKT lower bound: 2   2020-11-08
Name: IPO_DATE, dtype: datetime64[ns]
IKT upper bound: 2   2021-02-06
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
GBS: 3   2020-12-23
Name: IPO_DATE, dtype: datetime64[ns]
GBS lower bound: 3   2020-11-08
Name: IPO_DATE, dtype: datetime64[ns]
GBS upper bound: 3   2021-02-06
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
HCAR: 4   2020-12-23
Name: IPO_DATE, dtype: datetime64[ns]
HCAR l

GHVI: 39   2020-12-11
Name: IPO_DATE, dtype: datetime64[ns]
GHVI lower bound: 39   2020-10-27
Name: IPO_DATE, dtype: datetime64[ns]
GHVI upper bound: 39   2021-01-25
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
TVAC: 40   2020-12-11
Name: IPO_DATE, dtype: datetime64[ns]
TVAC lower bound: 40   2020-10-27
Name: IPO_DATE, dtype: datetime64[ns]
TVAC upper bound: 40   2021-01-25
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
CBAH: 41   2020-12-11
Name: IPO_DATE, dtype: datetime64[ns]
CBAH lower bound: 41   2020-10-27
Name: IPO_DATE, dtype: datetime64[ns]
CBAH upper bound: 41   2021-01-25
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
VVOS: 42   2020-12-11
Name: IPO_DATE, dtype: datetime64[ns]
VVOS lower bound: 42   2020-10-27
Name: IPO_DATE, dtype: datetime64[ns]
VVOS upper bound: 42   2021-01-25
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
CERT: 43   2020-12-11
Name: IPO_DATE, dtype:

----------------------------------
SV: 80   2020-11-24
Name: IPO_DATE, dtype: datetime64[ns]
SV lower bound: 80   2020-10-10
Name: IPO_DATE, dtype: datetime64[ns]
SV upper bound: 80   2021-01-08
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
VMAR: 81   2020-11-24
Name: IPO_DATE, dtype: datetime64[ns]
VMAR lower bound: 81   2020-10-10
Name: IPO_DATE, dtype: datetime64[ns]
VMAR upper bound: 81   2021-01-08
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
GNPK: 82   2020-11-24
Name: IPO_DATE, dtype: datetime64[ns]
GNPK lower bound: 82   2020-10-10
Name: IPO_DATE, dtype: datetime64[ns]
GNPK upper bound: 82   2021-01-08
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
VCVC: 83   2020-11-24
Name: IPO_DATE, dtype: datetime64[ns]
VCVC lower bound: 83   2020-10-10
Name: IPO_DATE, dtype: datetime64[ns]
VCVC upper bound: 83   2021-01-08
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
TINV: 84   2020

CONX: 118   2020-10-30
Name: IPO_DATE, dtype: datetime64[ns]
CONX lower bound: 118   2020-09-15
Name: IPO_DATE, dtype: datetime64[ns]
CONX upper bound: 118   2020-12-14
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
SQZ: 119   2020-10-30
Name: IPO_DATE, dtype: datetime64[ns]
SQZ lower bound: 119   2020-09-15
Name: IPO_DATE, dtype: datetime64[ns]
SQZ upper bound: 119   2020-12-14
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
JUPW: 120   2020-10-30
Name: IPO_DATE, dtype: datetime64[ns]
JUPW lower bound: 120   2020-09-15
Name: IPO_DATE, dtype: datetime64[ns]
JUPW upper bound: 120   2020-12-14
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
BHSE: 121   2020-10-30
Name: IPO_DATE, dtype: datetime64[ns]
BHSE lower bound: 121   2020-09-15
Name: IPO_DATE, dtype: datetime64[ns]
BHSE upper bound: 121   2020-12-14
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
AVIR: 122   2020-10-30
Name: IPO_DA

...

Name: IPO_DATE, dtype: datetime64[ns]
AIH lower bound: 524   2019-09-10
Name: IPO_DATE, dtype: datetime64[ns]
AIH upper bound: 524   2019-12-09
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
CABA: 525   2019-10-25
Name: IPO_DATE, dtype: datetime64[ns]
CABA lower bound: 525   2019-09-10
Name: IPO_DATE, dtype: datetime64[ns]
CABA upper bound: 525   2019-12-09
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
HAPP: 526   2019-10-25
Name: IPO_DATE, dtype: datetime64[ns]
HAPP lower bound: 526   2019-09-10
Name: IPO_DATE, dtype: datetime64[ns]
HAPP upper bound: 526   2019-12-09
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
BRP: 527   2019-10-24
Name: IPO_DATE, dtype: datetime64[ns]
BRP lower bound: 527   2019-09-09
Name: IPO_DATE, dtype: datetime64[ns]
BRP upper bound: 527   2019-12-08
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
GLEO: 528   2019-10-18
Name: IPO_DATE, dtype: datetime64[ns]

SNDL: 564   2019-08-01
Name: IPO_DATE, dtype: datetime64[ns]
SNDL lower bound: 564   2019-06-17
Name: IPO_DATE, dtype: datetime64[ns]
SNDL upper bound: 564   2019-09-15
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
DT: 565   2019-08-01
Name: IPO_DATE, dtype: datetime64[ns]
DT lower bound: 565   2019-06-17
Name: IPO_DATE, dtype: datetime64[ns]
DT upper bound: 565   2019-09-15
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
NFIN: 566   2019-07-31
Name: IPO_DATE, dtype: datetime64[ns]
NFIN lower bound: 566   2019-06-16
Name: IPO_DATE, dtype: datetime64[ns]
NFIN upper bound: 566   2019-09-14
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
FileNotFoundError occured on NFIN
----------------------------------
BORR: 567   2019-07-31
Name: IPO_DATE, dtype: datetime64[ns]
BORR lower bound: 567   2019-06-16
Name: IPO_DATE, dtype: datetime64[ns]
BORR upper bound: 567   2019-09-14
Name: IPO_DATE, dtype: datetime64[ns]
----

AKRO: 603   2019-06-20
Name: IPO_DATE, dtype: datetime64[ns]
AKRO lower bound: 603   2019-05-06
Name: IPO_DATE, dtype: datetime64[ns]
AKRO upper bound: 603   2019-08-04
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
PSNL: 604   2019-06-20
Name: IPO_DATE, dtype: datetime64[ns]
PSNL lower bound: 604   2019-05-06
Name: IPO_DATE, dtype: datetime64[ns]
PSNL upper bound: 604   2019-08-04
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
GO: 605   2019-06-20
Name: IPO_DATE, dtype: datetime64[ns]
GO lower bound: 605   2019-05-06
Name: IPO_DATE, dtype: datetime64[ns]
GO upper bound: 605   2019-08-04
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
BCEL: 606   2019-06-20
Name: IPO_DATE, dtype: datetime64[ns]
BCEL lower bound: 606   2019-05-06
Name: IPO_DATE, dtype: datetime64[ns]
BCEL upper bound: 606   2019-08-04
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
WORK: 607   2019-06-20
Name: IPO_DATE,

YJ: 643   2019-05-03
Name: IPO_DATE, dtype: datetime64[ns]
YJ lower bound: 643   2019-03-19
Name: IPO_DATE, dtype: datetime64[ns]
YJ upper bound: 643   2019-06-17
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
RRBI: 644   2019-05-03
Name: IPO_DATE, dtype: datetime64[ns]
RRBI lower bound: 644   2019-03-19
Name: IPO_DATE, dtype: datetime64[ns]
RRBI upper bound: 644   2019-06-17
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
ATIF: 645   2019-05-03
Name: IPO_DATE, dtype: datetime64[ns]
ATIF lower bound: 645   2019-03-19
Name: IPO_DATE, dtype: datetime64[ns]
ATIF upper bound: 645   2019-06-17
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
BYND: 646   2019-05-02
Name: IPO_DATE, dtype: datetime64[ns]
BYND lower bound: 646   2019-03-18
Name: IPO_DATE, dtype: datetime64[ns]
BYND upper bound: 646   2019-06-16
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
SY: 647   2019-05-02
Name: IPO_DATE, d

THCB: 681   2019-03-05
Name: IPO_DATE, dtype: datetime64[ns]
THCB lower bound: 681   2019-01-19
Name: IPO_DATE, dtype: datetime64[ns]
THCB upper bound: 681   2019-04-19
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
HCAC: 682   2019-03-01
Name: IPO_DATE, dtype: datetime64[ns]
HCAC lower bound: 682   2019-01-15
Name: IPO_DATE, dtype: datetime64[ns]
HCAC upper bound: 682   2019-04-15
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
FileNotFoundError occured on HCAC
----------------------------------
KLDO: 683   2019-02-28
Name: IPO_DATE, dtype: datetime64[ns]
KLDO lower bound: 683   2019-01-14
Name: IPO_DATE, dtype: datetime64[ns]
KLDO upper bound: 683   2019-04-14
Name: IPO_DATE, dtype: datetime64[ns]
----------------------------------
SHLL: 684   2019-02-28
Name: IPO_DATE, dtype: datetime64[ns]
SHLL lower bound: 684   2019-01-14
Name: IPO_DATE, dtype: datetime64[ns]
SHLL upper bound: 684   2019-04-14
Name: IPO_DATE, dtype: datetime64[ns

In [7]:
# Print length of FileNotFoundError List
len(FileNotFoundError_List)

48

This is expected. During the process of retrieving the quarterly cash flow on Alpha Vantage for each symbol, 48 our of 710 symbols on the IPO_List had occured a KeyError where quarterlyReports were not found in the API response.